# 미국 나스닥 데이터 분석
## 주제 : 확정
# 순서
0. 패키지불러오기<br>
1.데이터 프레임 작성<br>
-파일목록 다 들고오기<br>
-들고온 파일 목록을 읽어서 데이터 프레임<br>
-데이터 프레임 결합<br>
2.시계열 데이터 시각화<br>
3.시계열 데이터 분리<br>
4.증가률 예측

In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'Malgun Gothic'
# from fbprophet import Prophet

# Set the working directory


# Get the list of files
files = os.listdir("nasdaq_stock")

# Read and process the stock data
stocks = pd.concat([pd.read_csv(file).assign(name=file.replace(".csv", "").replace("nasdaq_stock/", "")) for file in files])
stocks = stocks.rename(columns=str.lower)

# Read the nasdaq_stock_names.csv file
df = pd.read_csv("nasdaq_stock_names.csv")

# Merge stocks and df dataframes
stocks = pd.merge(stocks, df, left_on="name", right_on="stock_symbol")

# Visualize the opening prices over time
end_labels = stocks.groupby("company").last().sort_values("open", ascending=False).iloc[[0, 1, 2, 11, 12, 13], [3, 1]]

fig, ax = plt.subplots()
stocks.groupby("company").plot(x="date", y="open", ax=ax, legend=False)
ax2 = ax.twinx()
ax2.set_ylim(ax.get_ylim())
ax2.set_yticks(end_labels["open"])
ax2.set_yticklabels(end_labels["company"])
ax.set_xlabel("")
ax.set_ylabel("Open")
ax.set_title("주요 회사의 시작가격")
plt.show()

# Separate line plots for top 3 and bottom 3 companies
top3_companies = end_labels.iloc[:3, 1].tolist()
bottom3_companies = end_labels.iloc[3:, 1].tolist()

top3_stocks = stocks[stocks["company"].isin(top3_companies)]
bottom3_stocks = stocks[stocks["company"].isin(bottom3_companies)]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
for company in top3_companies:
    top3_stocks[top3_stocks["company"] == company].plot(x="date", y="open", ax=ax1, label=company)
ax1.legend(loc="upper left")
ax1.set_title("Top 3")
ax1.set_xlabel("")
ax1.set_ylabel("Open")

for company in bottom3_companies:
    bottom3_stocks[bottom3_stocks["company"] == company].plot(x="date", y="open", ax=ax2, label=company)
ax2.legend(loc="upper left")
ax2.set_title("Bottom 3")
ax2.set_xlabel("Date")
ax2.set_ylabel("Open")

plt.tight_layout()
plt.show()

# Time series analysis for AAPL stock
aapl = stocks[stocks["name"] == "AAPL"][["date", "open"]]
aapl["diff"] = aapl["open"].diff()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))

ax1.plot(aapl["date"], aapl["diff"], color="steelblue", alpha=0.7, marker="o")
ax1.set_xlabel("Date")
ax1.set_ylabel("Difference")
ax1.set_title("One Day Return")

ax2.hist(aapl["diff"], bins=50, color="steelblue", edgecolor="black")
ax2.set_xlabel("Difference")
ax2.set_ylabel("Frequency")

plt.tight_layout()
plt.show()

# Time series forecasting with Prophet
# aapl_prophet = aapl.rename(columns={"date": "ds", "open": "y"})/
# m_aapl = Prophet()
# m_aapl.fit(aapl_prophet)

# future = m_aapl.make_future_dataframe(periods=140)
# forecast = m_aapl.predict(future)

# fig = m_aapl.plot(forecast)
# plt.show()

# fig = m_aapl.plot_components(forecast)
# plt.show()


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'nasdaq_stock'